In [1]:
import json
from keras.applications import InceptionV3 as inceptionv3

Using TensorFlow backend.


In [2]:
img_rows = 299
img_cols = 299

In [3]:
model = inceptionv3(weights = 'imagenet',
                   include_top = False,
                   input_shape = (img_rows,img_cols,3))

In [4]:
for layer in model.layers:
    layer.trainable = False
for (i,layer) in enumerate(model.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 Activation False
4 Conv2D False
5 BatchNormalization False
6 Activation False
7 Conv2D False
8 BatchNormalization False
9 Activation False
10 MaxPooling2D False
11 Conv2D False
12 BatchNormalization False
13 Activation False
14 Conv2D False
15 BatchNormalization False
16 Activation False
17 MaxPooling2D False
18 Conv2D False
19 BatchNormalization False
20 Activation False
21 Conv2D False
22 Conv2D False
23 BatchNormalization False
24 BatchNormalization False
25 Activation False
26 Activation False
27 AveragePooling2D False
28 Conv2D False
29 Conv2D False
30 Conv2D False
31 Conv2D False
32 BatchNormalization False
33 BatchNormalization False
34 BatchNormalization False
35 BatchNormalization False
36 Activation False
37 Activation False
38 Activation False
39 Activation False
40 Concatenate False
41 Conv2D False
42 BatchNormalization False
43 Activation False
44 Conv2D False
45 Conv2D False
46 BatchNormalization False
47 Batc

In [5]:
def addTopModel(bottom_model, num_class, D=256):
    top_model = bottom_model.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(D, activation="relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation="softmax")(top_model)
    return top_model

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [7]:
file = open('config.json')
data = json.load(file)

In [8]:
num_classes = int(data['num_classes'])

In [9]:
FC_Head = addTopModel(model, num_classes)

In [10]:
new_model = Model(inputs=model.input, outputs=FC_Head)
print(new_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = data['train_data_dir']
val_data_dir = data['val_data_dir']

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

In [13]:
train_batchsize = data['train_batchsize']
val_batchsize = data['val_batchsize']

In [14]:
train_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=train_batchsize,
    class_mode='categorical')

val_gen = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=val_batchsize,
    class_mode='categorical',
    shuffle=False)

Found 740 images belonging to 4 classes.
Found 151 images belonging to 4 classes.


In [15]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
import PIL

In [16]:
chk_pt = data['ModelCheckPoint']
checkpoint = ModelCheckpoint(data['model_name'],
                             monitor=chk_pt['monitor'],
                             mode=chk_pt['mode'],
                             save_best_only=chk_pt['save_best_only'],
                             verbose=chk_pt['verbose'])

In [17]:
es = data['EarlyStopping']
earlystop = EarlyStopping(monitor=es['monitor'],
                          min_delta=es['min_delta'],
                          patience=es['patience'],
                          verbose=es['verbose'],
                          restore_best_weights=es['restore_best_weights'])

In [18]:
callbacks = [earlystop, checkpoint]

new_model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(lr=0.001),
                  metrics=['accuracy'])

In [19]:
train_samples = data['train_samples']
val_samples = data['validation_samples']
epochs = data['num_epochs']
batch_size = data['batch_size']

In [20]:
history = new_model.fit_generator(
    train_gen,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_gen,
    validation_steps=val_samples // batch_size)
new_model.save(data['model_name'])

Epoch 1/10
74/74 [==============================] - 386s 5s/step - loss: 17.7248 - accuracy: 0.3676 - val_loss: 0.5044 - val_accuracy: 0.3677

Epoch 00001: val_loss improved from inf to 0.50436, saving model to model.hd5
Epoch 2/10
12/74 [===>..........................] - ETA: 4:18 - loss: 1.6593 - accuracy: 0.3875

KeyboardInterrupt: 